### 題目
    由手動的技巧來進行選擇 feature，以 Bank 為範例進行資料前處理，整理後的資料，以 CSV 的格式輸出，處理之後的資料集，以 linear regression （至少包含線性以及非線性的兩種機制）進行預測。
 

In [332]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# 讀取資料集
data = pd.read_csv("../../data/bank/bank-full.csv", sep=";", header="infer")
print("資料集大小:", data.shape, "\n")
print("欄位:", data.columns.tolist(), "\n")
print(data.head(), "\n")

data.isnull().sum()

資料集大小: (45211, 17) 

欄位: ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y'] 

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unk

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

### 處理二元類別資料

In [333]:
## 處理二元類別資料
data.y = data.y.map({"yes": 1, "no": 0})
data.default = data.default.map({"yes": 1, "no": 0})
data.housing = data.housing.map({"yes": 1, "no": 0})
data.loan = data.loan.map({"yes": 1, "no": 0})

### 處理 Unknown 欄位

In [334]:
# 檢查各欄位有多少 "unknown" 值
unknown_counts = {}
for col in data.columns:
    # 字串欄位才需要檢查 "unknown"
    if data[col].dtype == "object":
        count = (data[col] == "unknown").sum()
        if count > 0:
            unknown_counts[col] = count

# 顯示結果
if unknown_counts:
    print("含有 'unknown' 值的欄位:")
    for col, count in unknown_counts.items():
        print(f"{col}: {count} 筆")
else:
    print("沒有欄位含 'unknown' 值")

# 處理 'unknown' 值欄位
# job, education 以眾數、平均數取代 'unknown' 值, poutcome 因 unknown 筆數很多，將 unknown 視為一類別
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values="unknown", strategy="most_frequent")

# job 的 unknown 僅占約 0.8%，透過眾數填補
nan_columns = ["job"]
data[nan_columns] = imp.fit_transform(data[nan_columns])

含有 'unknown' 值的欄位:
job: 288 筆
education: 1857 筆
contact: 13020 筆
poutcome: 36959 筆


### Feature Selection
    👍保留 age ，原因: 年齡影響理財習慣、風險承受度，與金融產品接受度高度相關。
    👍保留 job ，原因: 職業反映收入、穩定性及社經地位。
    👍保留 marital ，原因: 婚姻狀態影響家庭責任與消費模式。
    👍保留 education ，原因: 教育程度影響理財知識與產品接受度。
    ⚠️稍後觀察 default 的變藝術 ，因為幾乎所有值為 "no"
    👍保留 balance ，原因: 存款餘額直接反映財務能力。
    👍保留 housing ，原因: 扛房貸可能沒錢定存。
    👍保留 loan ，原因: 扛貸款也可能影響定存能力。
    👍保留 contact ，原因: 有聯絡客戶可能比較會關注金融商品。
    ❌刪除 day、month 欄位，因為具有時序性，僅為行銷聯絡日期，與客戶財務行為無關。
    ❌刪除 duration 欄位，因為在真實預測情境中，無法事先得知通話持續時間。
    👍保留 campaign、pdays、previous 欄位，因為這些欄位反映過去行銷接觸頻率與效果，對預測有幫助。
    👍保留 poutcome 欄位，因為過去行銷結果對預測有幫助。
    
    ⭕結論: 刪除 default、day、month、duration 欄位

In [335]:
data.drop(["day", "month", "duration"], axis=1, inplace=True)

# pdays 比起上次聯繫這個資訊，銀行可能更重視是否曾經聯繫過客戶
# 因此將 pdays 轉成二元欄位 contacted
data["contacted"] = np.where(data["pdays"] > 0, 1, 0)
data.drop("pdays", axis=1, inplace=True)

print(data["default"].var())
data.drop("default", axis=1, inplace=True)

data.head()

0.0177020201735023


,age,job,marital,education,balance,housing,loan,contact,campaign,previous,poutcome,y,contacted
0,58,management,married,tertiary,2143,1,0,unknown,1,0,unknown,0,0
1,44,technician,single,secondary,29,1,0,unknown,1,0,unknown,0,0
2,33,entrepreneur,married,secondary,2,1,1,unknown,1,0,unknown,0,0
3,47,blue-collar,married,unknown,1506,1,0,unknown,1,0,unknown,0,0
4,33,blue-collar,single,unknown,1,0,0,unknown,1,0,unknown,0,0


In [336]:
categorical_columns = data.select_dtypes(include=["object"]).columns
print("類別欄位", categorical_columns)

# 所有的數值欄位的名稱
numeric_cols_index = data.select_dtypes(include=["int64", "float64"]).columns

numeric_cols = numeric_cols_index[data[numeric_cols_index].nunique() > 2].tolist()
binary_cols = numeric_cols_index[data[numeric_cols_index].nunique() == 2].tolist()

print("多值數值欄位 (nunique > 2):", numeric_cols)
print("二元欄位 (nunique = 2):", binary_cols)

類別欄位 Index(['job', 'marital', 'education', 'contact', 'poutcome'], dtype='object')
多值數值欄位 (nunique > 2): ['age', 'balance', 'campaign', 'previous']
二元欄位 (nunique = 2): ['housing', 'loan', 'y', 'contacted']


### One-Hot Encoding

In [337]:
from sklearn.preprocessing import OrdinalEncoder

# education、poutcome 具有順序性，使用 Ordinal Encoding
education_order = [
    "unknown",  # 0
    "primary",  # 1
    "secondary",  # 2
    "tertiary",  # 3
]

oe = OrdinalEncoder(categories=[education_order])
data["education"] = oe.fit_transform(data[["education"]])

poutcome_order = [
    "unknown",  # 0
    "failure",  # 1
    "other",  # 2
    "success",  # 3
]

oe = OrdinalEncoder(categories=[poutcome_order])
data["poutcome"] = oe.fit_transform(data[["poutcome"]])

# 再次確認類別欄位
categorical_columns = data.select_dtypes(include=["object"]).columns
print("剩餘的類別欄位:", categorical_columns)

# 剩餘的類別欄位使用 One-Hot Encoding
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop="first")
data_encoded = ohe.fit_transform(data[categorical_columns])
data_encoded = pd.DataFrame(
    data_encoded,
    columns=ohe.get_feature_names_out(categorical_columns),
)
data = pd.concat([data, data_encoded], axis=1)
data.drop(categorical_columns, axis=1, inplace=True)

data.head()

剩餘的類別欄位: Index(['job', 'marital', 'contact'], dtype='object')


,age,education,balance,housing,loan,campaign,previous,poutcome,y,contacted,...,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,contact_telephone,contact_unknown
0,58,3.0,2143,1,0,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,44,2.0,29,1,0,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,33,2.0,2,1,1,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,47,0.0,1506,1,0,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,33,0.0,1,0,0,1,0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


### 變異數特徵篩選

In [338]:
from sklearn.feature_selection import VarianceThreshold

# 分離特徵與標籤
X = data.drop("y", axis=1)
y = data["y"]

variances = X.var().sort_values()
print(variances)

# 移除變異量低於 0.03 的特徵
low_variance_features = variances[variances < 0.03]
print(low_variance_features)

vt = VarianceThreshold(threshold=0.021)
X_filtered = vt.fit_transform(X)
selected_columns = X.columns[vt.get_support()]

X = pd.DataFrame(X_filtered, columns=selected_columns)
print("篩選後的特徵欄位:", X.columns.tolist())

job_student          2.031717e-02
job_housemaid        2.667531e-02
job_unemployed       2.799042e-02
job_entrepreneur     3.180916e-02
job_self-employed    3.370611e-02
job_retired          4.756972e-02
contact_telephone    6.014627e-02
job_services         8.344015e-02
loan                 1.345569e-01
job_technician       1.398019e-01
contacted            1.492812e-01
job_management       1.654372e-01
job_blue-collar      1.725126e-01
marital_single       2.028702e-01
contact_unknown      2.050533e-01
marital_married      2.396149e-01
housing              2.468876e-01
poutcome             4.878594e-01
education            6.063797e-01
previous             5.305841e+00
campaign             9.597733e+00
age                  1.127581e+02
balance              9.270599e+06
dtype: float64
job_student       0.020317
job_housemaid     0.026675
job_unemployed    0.027990
dtype: float64
篩選後的特徵欄位: ['age', 'education', 'balance', 'housing', 'loan', 'campaign', 'previous', 'poutcome', 'contacted

### 標準化

In [339]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
X.head()

,age,education,balance,housing,loan,campaign,previous,poutcome,contacted,job_blue-collar,...,job_management,job_retired,job_self-employed,job_services,job_technician,job_unemployed,marital_married,marital_single,contact_telephone,contact_unknown
0,1.606965,3.0,0.256419,1.0,0.0,-0.569351,-0.25194,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.288529,2.0,-0.437895,1.0,0.0,-0.569351,-0.25194,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,-0.747384,2.0,-0.446762,1.0,1.0,-0.569351,-0.25194,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.571051,0.0,0.047205,1.0,0.0,-0.569351,-0.25194,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,-0.747384,0.0,-0.447091,0.0,0.0,-0.569351,-0.25194,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [340]:
## 產出 csv 檔案
processed_data = X.copy()
processed_data["y"] = y

# 輸出 CSV
processed_data.to_csv(
    "../../output/week7_hw_data_processing/feature_select_by_myself.csv", index=False
)
print(
    "✅ CSV 已輸出到 ../../output/week7_hw_data_processing/feature_select_by_myself.csv"
)

✅ CSV 已輸出到 ../../output/week7_hw_data_processing/feature_select_by_myself.csv


### 預測1. Linear Regression

In [341]:
# Linear Rregression

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

# 讀取前處理後的資料
data = pd.read_csv("../../output/week7_hw_data_processing/feature_select_by_myself.csv")
X = data.drop(columns=["y"]).values
y = data["y"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1000
)
lr = LinearRegression()
lr.fit(X_train, y_train)
print("預測值", lr.predict(X_test))
print("實際值", y_test)
print("r2 socre", lr.score(X_test, y_test))

預測值 [0.01103602 0.06266595 0.12542766 ... 0.17712698 0.02450225 0.00438538]
實際值 [0 0 0 ... 0 0 0]
r2 socre 0.12282797111681054


In [342]:
# Linear Regression with k-fold cross validation

scores = cross_val_score(lr, X, y, cv=7, scoring="r2")
print("Cross-validation scores:", scores)

Cross-validation scores: [-0.05653131 -0.07740829 -0.09060994 -0.24342791 -0.06022172 -0.02787458
 -0.28776528]


In [343]:
# Ridge Regression
from sklearn.linear_model import RidgeCV

rgs = RidgeCV(
    alphas=(
        1.0,
        0.5,
        0.1,
        0.01,
    ),
)

rgs.fit(X_train, y_train)
print("Ridge scores", rgs.score(X_test, y_test))
print("Ridge best alpha", rgs.alpha_)

rg_scores = cross_val_score(rgs, X, y, cv=7, scoring="r2")
print("Ridge Cross-validation scores:", rg_scores)

Ridge scores 0.12282466559090077
Ridge best alpha 1.0
Ridge Cross-validation scores: [-0.0565183  -0.07741149 -0.09057633 -0.24337301 -0.06023058 -0.02789867
 -0.28792042]


In [344]:
# Lasso Regression

from sklearn.linear_model import LassoCV

lss = LassoCV(
    alphas=(1.0, 0.5, 0.1, 0.01, 0.001),
)

lss.fit(X_train, y_train)
print("Lasso scores", lss.score(X_test, y_test))
print("Lasso best alpha", lss.alpha_)
ls_scores = cross_val_score(lss, X, y, cv=7, scoring="r2")
print("Lasso Cross-validation scores:", ls_scores)

Lasso scores 0.12023411902041536
Lasso best alpha 0.001
Lasso Cross-validation scores: [-0.04443307 -0.07622161 -0.08148542 -0.21899765 -0.06446247 -0.00089073
 -0.37296232]


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel

pf = PolynomialFeatures(degree=2)
Xp = pf.fit_transform(X)
print("多項式特徵後的形狀:", Xp.shape)

lr = LinearRegression()
lr.fit(Xp, y)
print("原始 R²:", lr.score(Xp, y))

# Step2: SelectFromModel 篩選特徵
sm = SelectFromModel(lr, threshold=0.001)
Xt = sm.fit_transform(Xp, y)
sm.estimator_.score(Xp, y)

lr2 = LinearRegression().fit(Xt, y)
print("篩選後 R²:", lr2.score(Xt, y))

多項式特徵後的形狀: (45211, 276)
原始 R²: 0.15375729814187922
篩選後 R²: 0.15363166909788772
